# DX트랙 5차 프로젝트 
## API를 이용하여 공공 데이터 가져오기

 
 - <font size =+1> 공공데이터포털(https://data.go.kr)에서 제공되는 [국립중앙의료원 - 전국응급의료기관 조회 서비스]에서 필요한 자료를 API를 통해 가져옵니다.
 - <font size =+1> 회원 가입 후 해당 자료에 [활용 신청]을 한 뒤, 마이페이지에서 일반 인증키(encoding)을 확인합니다. (PDF 교재 참고)
 - <font size =+1> 대구광역시 중심으로 데이터 추출 
 

 * 사전 확인 사항 
   * haversine 설치
   * numpy 버전: python 3.9인 경우 == 1.21.0, python 3.10인 경우 == 1.23.5
   * pandas 버전 > 1.3
   * tabulate 설치 확인

In [1]:
# !pip install haversine --user
# !pip install numpy==1.21.0 --upgrade --user # python 3.9인 경우
# !pip install numpy==1.23.5 --upgrade --user # python 3.10인 경우
# !pip install pandas==1.5.3 --upgrade --user
# !pip install tabulate --user

In [2]:
## windows 도스창 기준입니다.
## haversine 설치 확인
!pip list | find "haversine"

haversine                     2.8.0


In [3]:
## windows 도스창 기준입니다.
## 파이썬 버전 확인
!python --version

Python 3.9.13


In [4]:
## windows 도스창 기준입니다.
## numpy 버전 확인
!pip list | find "numpy"

numpy                         1.21.5
numpydoc                      1.4.0


In [5]:
## windows 도스창 기준입니다.
## pandas 버전 확인
!pip list | find "pandas"

pandas                        1.4.4


In [6]:
## windows 도스창 기준입니다.
## tabulate 설치 확인
!pip list | find "tabulate"

tabulate                      0.8.10


In [7]:
# 필요한 라이브러리 불러오기

import pandas as pd
import numpy as np
from urllib.parse import quote

In [8]:
#  [국립중앙의료원 - 전국응급의료기관 조회 서비스] 활용을 위한 개인 일반 인증키(Encoding) 저장

key = "1DQKrkMBrPphK39z1SyNfz5dSesh5VlV%2F6VHMdBgr85Vq%2Fu3YRpKnu9V2phrBdVzYDRdAmZ10rVxDr4kPLbUWw%3D%3D"

# city = 대구광역시, 인코딩 필요
city = quote("대구광역시")

In [9]:
import ssl
from urllib.request import urlopen
context=ssl.create_default_context()
context.set_ciphers("DEFAULT")


### 1) 대구광역시 응급의료기관 목록정보 조회하기

 * 주소 (대구광역시)를 기준으로 응급 의료 기관 목록 정보 조회
 * 기관ID, 주소, 기관명, 응급실전화, 병원 위치(경도/위도) 정보를 확인

In [10]:
# 대구광역시내이 응급의료기관 목록정보 조회하기
# url : https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire
# 인증키 지정 : serviceKey = key
# 도시 지정 : Q0=city
# 출력 행수 : numofRows=100 
# page 번호 : pageNo = 1


url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire' + '?serviceKey=' + key + '&Q0=' + city + '&pageNo=1&numOfRows=100'

# emrList = pd.read_xml(url, xpath='/response/body/items/item')
result = urlopen(url, context=context)
emrList = pd.read_xml(result, xpath='.//item')

emrList.head()

,dutyAddr,dutyEmcls,dutyEmclsName,dutyName,dutyTel1,dutyTel3,hpid,phpid,rnum,wgs84Lat,wgs84Lon
0,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",G007,지역응급의료기관,(재)미리내천주성삼성직수도회천주성삼병원,053-790-1000,053-790-0555,A1300081,A1300081,1,35.840225,128.705733
1,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",G007,지역응급의료기관,강남병원,053-980-9000,053-980-9018,A1300018,A1300018,2,35.883009,128.663456
2,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",G001,권역응급의료센터,경북대학교병원,053-200-5114,053-200-5100,A1300002,A1300002,3,35.866235,128.604315
3,"대구광역시 중구 달성로 56, 계명대학교대구동산병원 (동산동)",G007,지역응급의료기관,계명대학교대구동산병원,053-250-8114,053-250-8481,A1303386,A1303386,4,35.869460,128.582931
4,대구광역시 달서구 달구벌대로 1035 (신당동),G006,지역응급의료센터,계명대학교동산병원,1577-6622,053-258-6306,A1300003,A1300003,5,35.853887,128.480130


In [11]:
## ermList 중 [hpid, dutyName, dutyAddr, dutyTel3, wgs84Lat, wgs84Lon] 컬럼 활용
## solution_df 에 저장하기

solution_df = emrList[['hpid', 'dutyName', 'dutyAddr', 'dutyTel3', 'wgs84Lat', 'wgs84Lon']].copy()
solution_df.head()

,hpid,dutyName,dutyAddr,dutyTel3,wgs84Lat,wgs84Lon
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9018,35.883009,128.663456
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315
3,A1303386,계명대학교대구동산병원,"대구광역시 중구 달성로 56, 계명대학교대구동산병원 (동산동)",053-250-8481,35.869460,128.582931
4,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6306,35.853887,128.480130


### 2) 응급의료기관별 기본정보 조회 - 응급실수, 수술실 수 확인

 * <b>기관ID</b>를 기준으로 응급실, 수술실 정보 등 응급의료기관 기본 정보 조회 가능
 * 응급실수 (hperyn)와 수술실 수(hpopyn) 정보를 확인

In [12]:
# 대구광역시내이 응급의료기관 목록정보 조회하기
# url : https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytBassInfoInqire
# 인증키 지정 : serviceKey = key
# 응급의료기관 지정 : HPID = hpid
# 응급의료기관 하나씩 조회해야 함

# 응급의료기관에 대한 hpid값을 list로 추출
hpidList = list(solution_df['hpid'])

# 각 응급의료기관의 응급실 병상수(hperyn), 수술실 수(hpopyn) 를 저장하기 위한 빈 리스트 생성
hperynList = []
hpopynList = []

# for문을 이용하여 hpid 하나씩 조회하고, hperyn, hpopy 정보를 각 hperynList, hpopynList 리스트에 저장

for hpid in hpidList:
    
    url_basic = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytBassInfoInqire' + '?serviceKey=' + key + '&HPID=' + hpid 
    result = urlopen(url_basic, context=context)
    df_temp = pd.read_xml(result, xpath='.//item')

    # hperyn, hpopyn이 필수가 아닌 옵션값으로, 값이 전달되지 않을 때를 대비하여 예외처리로 0의 값을 입력
    
    if 'hperyn' in list(df_temp.columns) :
        hperynList.append(df_temp['hperyn'][0])
    else:
        hperynList.append(0)
    
    if 'hpopyn' in list(df_temp.columns) :
        hpopynList.append(df_temp['hpopyn'][0])
    else:
        hpopynList.append(0)
    
    

In [13]:
# 기존 solution_df에 응급실수, 수술실 수 정보 추가하기

solution_df['hperyn'] = hperynList
solution_df['hpopyn'] = hpopynList
solution_df.head()

,hpid,dutyName,dutyAddr,dutyTel3,wgs84Lat,wgs84Lon,hperyn,hpopyn
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9018,35.883009,128.663456,10,4
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16
3,A1303386,계명대학교대구동산병원,"대구광역시 중구 달성로 56, 계명대학교대구동산병원 (동산동)",053-250-8481,35.869460,128.582931,0,0
4,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6306,35.853887,128.480130,28,24


In [14]:
## hperyn, hpopyn값이 0인 병원 삭제
solution_df = solution_df[solution_df['hpopyn']>0]
solution_df = solution_df[solution_df['hperyn']>0]

In [15]:
## solution_df 를 파일로 저장
## 3일차 프로토타입에는 파일에서 불러오기로 진행

solution_df.to_csv('./daegu_hospital_list.csv', index=False)

In [252]:
# 최종결과물: 응급실 수 , 수술실 수가 있는 병워 최종확인
solution_df=pd.read_csv('./daegu_hospital_list.csv')
solution_df.head()

,hpid,dutyName,dutyAddr,dutyTel3,wgs84Lat,wgs84Lon,hperyn,hpopyn
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9018,35.883009,128.663456,10,4
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6306,35.853887,128.480130,28,24
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4


### 3) 응급의료기관별 실시간 가용 병상 조회

 * 주소(대구광역시)를 기준으로 응급실 실시간 가용병상정보 등을 조회
 * 실시간 응급실(hvec), 수술실(hvoc) 가용 수량 정보 확인

In [156]:
# 응급실 실시간 가용병상 조회
# url : https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire
# 인증키 지정 : serviceKey = key
# 도시 지정 : STAGE1=city
# 출력 행수 : numofRows=100 
# page 번호 : pageNo = 1

#  [국립중앙의료원 - 전국응급의료기관 조회 서비스] 활용을 위한 개인 일반 인증키(Encoding) 저장
key = "1DQKrkMBrPphK39z1SyNfz5dSesh5VlV%2F6VHMdBgr85Vq%2Fu3YRpKnu9V2phrBdVzYDRdAmZ10rVxDr4kPLbUWw%3D%3D"

# city = 대구광역시, 인코딩 필요
city = quote("대구광역시")

url_realtime = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire'+'?serviceKey=' + key + '&STAGE1=' + city + '&pageNo=1&numOfRows=100' 

result = urlopen(url_realtime, context=context)
emrRealtime = pd.read_xml(result, xpath='.//item')

emrRealtime.head()

,dutyName,dutyTel3,hpid,hv27,hv30,hv5,hv7,hvamyn,hvangioayn,hvcrrtayn,...,hvs48,hvs49,hv38,hv39,hv9,hvccc,hvs14,hvs16,hvs21,hvs23
0,드림종합병원,053-640-8575,A1300034,1.0,1.0,Y,Y,Y,Y,N1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,대구가톨릭대학교병원,053-650-3025,A1300004,NaN,0.0,Y,Y,Y,Y,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,대구굿모닝병원,053-620-9119,A1300076,NaN,1.0,Y,Y,Y,Y,N1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,영남대학교병원,053-620-3177,A1300001,NaN,0.0,Y,Y,Y,Y,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,의료법인구의료재단구병원,053-557-7119,A1300006,NaN,1.0,Y,None,Y,N1,N1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
## emrRealtime 중 [hpid, hvec, hvoc] 컬럼 활용
# hvec : 응급실
# hvoc : 수술실
tmp=emrRealtime[['hpid', 'hvec', 'hvoc']]
emrRealtime[['hpid', 'hvec', 'hvoc']].head()

,hpid,hvec,hvoc
0,A1300034,7,3
1,A1300004,3,9
2,A1300076,7,2
3,A1300001,-1,9
4,A1300006,10,6


In [158]:
# 기존 dataframe에 emrRealtime[['hpid', 'hvec', 'hvoc']] 정보 합치기

solution_df = pd.merge(solution_df,tmp, on='hpid', how='left')
solution_df.head()

,hpid,dutyName,dutyAddr,dutyTel3,wgs84Lat,wgs84Lon,hperyn,hpopyn,hvec,hvoc
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,2
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9018,35.883009,128.663456,10,4,13,7
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,-12,7
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6306,35.853887,128.480130,28,24,0,6
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,4


In [159]:
solution_df.isnull().sum()

hpid        0
dutyName    0
dutyAddr    0
dutyTel3    0
wgs84Lat    0
wgs84Lon    0
hperyn      0
hpopyn      0
hvec        0
hvoc        0
dtype: int64

### 4) 응급의료기관별 중증질환자 수용 가능 여부 조회

 * 주소(대구광역시)를 기준으로 중증질환자 수용 가능 여부를 조회 (실시간 정보)

In [160]:
# 실시간 중증질환자 수용 가능 병원 조회
# url : https://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire
# 인증키 지정 : serviceKey = key
# 도시 지정 : STAGE1=city
# 출력 행수 : numofRows=100 
# page 번호 : pageNo = 1

url_acpt = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire'+'?serviceKey=' + key + '&STAGE1=' + city + '&pageNo=1&numOfRows=100' 


result = urlopen(url_acpt, context=context)
emrAcpt = pd.read_xml(result, xpath='.//item') 

emrAcpt.head()

## 갑자기 컬럼명이 바뀜
## hpid가 아닌 dutyName 컬럼명으로 출력됨
## 추후 원복 시 삭제해야할 수 있음

emrAcpt = emrAcpt.rename(columns={"dutyName":"hpid"})

emrAcpt.head()

,hpid,MKioskTy1,MKioskTy10,MKioskTy11,MKioskTy12,MKioskTy13,MKioskTy14,MKioskTy15,MKioskTy16,MKioskTy17,...,MKioskTy5,MKioskTy6,MKioskTy7,MKioskTy8,MKioskTy9,MKioskTy10Msg,MKioskTy12Msg,MKioskTy14Msg,MKioskTy15Msg,MKioskTy27Msg
0,A1300034,정보미제공,Y,Y,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,...,정보미제공,정보미제공,Y,Y,Y,None,None,None,None,None
1,A1300004,Y,Y,Y,정보미제공,Y,정보미제공,Y,Y,Y,...,Y,Y,Y,Y,Y,21일 이상 출생아부터 가능,0세부터 가능,.,.,.
2,A1300076,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,...,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,None,None,None,None,None
3,A1300001,Y,정보미제공,Y,정보미제공,Y,정보미제공,Y,Y,Y,...,Y,Y,정보미제공,Y,정보미제공,전체 연령 가능,평일 외래만 가능,None,전체 체중아 가능,전체 연령 가능
4,A1300006,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,...,정보미제공,정보미제공,Y,정보미제공,Y,None,None,None,None,None


In [161]:
## emrAcpt 중 [hpid, MKioskTy1, MKioskTy2, MKioskTy3, MKioskTy4, MKioskTy5, MKioskTy7,MKioskTy8, MKioskTy10, MKioskTy11] 컬럼 확인
# 'MKioskTy1' : '뇌출혈수술', 
# 'MKioskTy2' : '뇌경색의재관류',
# 'MKioskTy3' : '심근경색의재관류',
# 'MKioskTy4' : '복부손상의수술',
# 'MKioskTy5' : '사지접합의수술',
# 'MKioskTy7' : '응급투석',
# 'MKioskTy8' : '조산산모',
# 'MKioskTy10' : '신생아',
# 'MKioskTy11' : '중증화상'
tmp=emrAcpt[['hpid', 'MKioskTy1', 'MKioskTy2', 'MKioskTy3', 'MKioskTy4', 'MKioskTy5', 'MKioskTy7','MKioskTy8', 'MKioskTy10','MKioskTy11']]
emrAcpt[['hpid', 'MKioskTy1', 'MKioskTy2', 'MKioskTy3', 'MKioskTy4', 'MKioskTy5', 'MKioskTy7','MKioskTy8', 'MKioskTy10','MKioskTy11']].head()

,hpid,MKioskTy1,MKioskTy2,MKioskTy3,MKioskTy4,MKioskTy5,MKioskTy7,MKioskTy8,MKioskTy10,MKioskTy11
0,A1300034,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,Y,Y,Y,Y
1,A1300004,Y,Y,Y,Y,Y,Y,Y,Y,Y
2,A1300076,정보미제공,Y,Y,Y,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공
3,A1300001,Y,Y,Y,Y,Y,정보미제공,Y,정보미제공,Y
4,A1300006,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,Y,정보미제공,정보미제공,정보미제공


In [162]:
# 기존 dataframe에 emrAcpt[['hpid', 'MKioskTy1', 'MKioskTy2', 'MKioskTy3', 'MKioskTy4', 'MKioskTy5', 'MKioskTy7','MKioskTy8','MKioskTy10', 'MKioskTy11']] 정보 합치기

solution_df = pd.merge(solution_df,tmp, on='hpid', how='left')
solution_df.head()

,hpid,dutyName,dutyAddr,dutyTel3,wgs84Lat,wgs84Lon,hperyn,hpopyn,hvec,hvoc,MKioskTy1,MKioskTy2,MKioskTy3,MKioskTy4,MKioskTy5,MKioskTy7,MKioskTy8,MKioskTy10,MKioskTy11
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,2,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,Y,정보미제공,정보미제공,Y
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9018,35.883009,128.663456,10,4,13,7,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,-12,7,Y,Y,Y,Y,Y,Y,Y,Y,Y
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6306,35.853887,128.480130,28,24,0,6,Y,Y,Y,Y,Y,Y,정보미제공,Y,Y
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,4,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,Y,Y,정보미제공,Y


In [163]:
## solution_df의 컬럼명 확인

solution_df.columns

Index(['hpid', 'dutyName', 'dutyAddr', 'dutyTel3', 'wgs84Lat', 'wgs84Lon',
       'hperyn', 'hpopyn', 'hvec', 'hvoc', 'MKioskTy1', 'MKioskTy2',
       'MKioskTy3', 'MKioskTy4', 'MKioskTy5', 'MKioskTy7', 'MKioskTy8',
       'MKioskTy10', 'MKioskTy11'],
      dtype='object')

In [164]:
# column명을 한글로 교체하기 위해 딕셔너리로 정의

column_change = {  'hpid' : '병원코드',
                   'dutyName' : '병원명',
                   'dutyAddr' : '주소',
                   'dutyTel3' : '응급연락처',
                   'wgs84Lat' : '위도', 
                   'wgs84Lon' : '경도',
                   'hperyn' : '응급실수',
                   'hpopyn' : '수술실수',
                   'hvec' : '가용응급실수',
                   'hvoc' : '가용수술실수',
                   'MKioskTy1' : '뇌출혈', 
                   'MKioskTy2' : '뇌경색',
                   'MKioskTy3' : '심근경색',
                   'MKioskTy4' : '복부손상',
                   'MKioskTy5' : '사지접합',
                   'MKioskTy7' : '응급투석',
                   'MKioskTy8' : '조산산모',
                   'MKioskTy10' : '신생아',
                   'MKioskTy11' : '중증화상',
                   
                   }

In [165]:
# column명 변경하기

solution_df = solution_df.rename(columns=column_change)
solution_df.head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,뇌출혈,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,2,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,Y,정보미제공,정보미제공,Y
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9018,35.883009,128.663456,10,4,13,7,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,-12,7,Y,Y,Y,Y,Y,Y,Y,Y,Y
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6306,35.853887,128.480130,28,24,0,6,Y,Y,Y,Y,Y,Y,정보미제공,Y,Y
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,4,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,Y,Y,정보미제공,Y


In [166]:
# column 값 중 정보 미제공은 N로 변경
solution_df = solution_df.replace('정보미제공','N')
solution_df.head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,뇌출혈,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,2,N,N,N,N,N,Y,N,N,Y
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9018,35.883009,128.663456,10,4,13,7,N,N,N,N,N,N,N,N,N
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,-12,7,Y,Y,Y,Y,Y,Y,Y,Y,Y
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6306,35.853887,128.480130,28,24,0,6,Y,Y,Y,Y,Y,Y,N,Y,Y
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,4,N,N,N,N,N,Y,Y,N,Y


### 5) 응급실, 수술실 자료 정비하기

 * 응급실/수술실 수가 0 이하의 수치가 나오는 비정상 사례 존재 --> 0이하의 수치는 모두 0으로 대체하여 정비
 * 응급실 가용율 (응급실 가용 수량 / 응급실수)을 계산
 * 응급실 포화도는 응급실 가용율에 따라 [불가/혼잡/보통/원활] 4단계로 구분

In [167]:
# 응급실수, 수술실 수, 가용응급실수, 가용수술실수 기초통계 확인

solution_df[(solution_df['가용응급실수']<0) | (solution_df['가용수술실수']<0)]

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,뇌출혈,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,-12,7,Y,Y,Y,Y,Y,Y,Y,Y,Y
7,A1300005,대구가톨릭대학교칠곡가톨릭병원,대구광역시 북구 칠곡중앙대로 440 (읍내동),053-320-2111,35.934880,128.549252,10,3,-1,3,N,N,N,N,N,N,N,N,N0
10,A1300009,대구파티마병원,대구광역시 동구 아양로 99 (신암동),053-940-7119,35.883934,128.623835,28,13,-12,5,Y,Y,Y,Y,N,Y,Y,Y,Y
14,A1300001,영남대학교병원,대구광역시 남구 현충로 170 (대명동),053-620-3177,35.847506,128.585131,30,19,-1,9,Y,Y,Y,Y,Y,N,Y,N,Y
16,A1300010,칠곡경북대학교병원,"대구광역시 북구 호국로 807, 칠곡경북대학교병원(임상실습동 1~11층 포함) (학정동)",053-200-2100,35.956083,128.564408,17,13,-2,14,Y,Y,Y,Y,N,Y,Y,N,Y


In [168]:
## 응급실수, 수술실 수가 0보다 작은 경우는 비정상 데이터로 추정
## 0보다 작은 수는 0으로 변경
solution_df.loc[(solution_df['가용응급실수']<0) | (solution_df['가용수술실수']<0), '가용응급실수']=0

In [169]:
solution_df.head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,뇌출혈,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,2,N,N,N,N,N,Y,N,N,Y
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9018,35.883009,128.663456,10,4,13,7,N,N,N,N,N,N,N,N,N
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,0,7,Y,Y,Y,Y,Y,Y,Y,Y,Y
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6306,35.853887,128.480130,28,24,0,6,Y,Y,Y,Y,Y,Y,N,Y,Y
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,4,N,N,N,N,N,Y,Y,N,Y


In [208]:
# 응급실 가용율 계산 : 가용 응급실수 / 응급실 수

solution_df['응급실가용율'] =  (solution_df['가용응급실수']/solution_df['응급실수'])*100

# 응급실 가용율이 1이 넘는 경우는 1로 대체
if solution_df['응급실가용율']>=100:
    solution_df['응급실가용율']==100

solution_df.head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,2,...,N,N,N,N,Y,N,N,Y,70.0,불가
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9018,35.883009,128.663456,10,4,13,7,...,N,N,N,N,N,N,N,N,130.0,불가
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,0,7,...,Y,Y,Y,Y,Y,Y,Y,Y,0.0,불가
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6306,35.853887,128.480130,28,24,0,6,...,Y,Y,Y,Y,Y,N,Y,Y,0.0,불가
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,4,...,N,N,N,N,Y,Y,N,Y,70.0,불가


In [209]:
# 응급실 가용율에 따라 포화도 분류
# 응급실 가용율 구분 단계 : ~10, 10~30, 30~60, 60~ 
# 포화도 명칭 : ['불가', '혼잡', '보통', '원활']
# pd.cut() 활용

solution_df['응급실포화도'] = pd.cut(solution_df['응급실가용율'],bins=[-1, 10, 30, 60, np.inf],labels=['불가','혼잡','보통','원활'])
solution_df.head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,2,...,N,N,N,N,Y,N,N,Y,70.0,원활
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9018,35.883009,128.663456,10,4,13,7,...,N,N,N,N,N,N,N,N,130.0,원활
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,0,7,...,Y,Y,Y,Y,Y,Y,Y,Y,0.0,불가
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6306,35.853887,128.480130,28,24,0,6,...,Y,Y,Y,Y,Y,N,Y,Y,0.0,불가
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,4,...,N,N,N,N,Y,Y,N,Y,70.0,원활


### 6) 병원 거리 구하기

 * 병원 위치(위도/경도)와 환자의 위치(임의의 위도/경도) 간의 거리 측정
 * haversine 라이브러리를 활용 (https://pypi.org/project/haversine/)
 * 병원 ~ 환자간의 거리를 ['2km이내', '5km이내', '10km이내', '10km이상'] 4단계로 구분

In [140]:
# 환자와 병원간의 거리 측정을 위한 라이브러리 설치
# !pip install haversine

In [141]:
# 라이브러리 import
from haversine import haversine

# 거리 측정 예시

haversine((35.209710,129.004751), (35.1631139, 129.1635509), unit = 'km') 

15.333288646032319

In [223]:
# 환자 위치 : 서문시장 내
patient= (35.8690760, 128.5811616)  #Latitude, Longitude

# 거리 계산

distance = []
distance_df = solution_df.copy()

for idx, row in distance_df.iterrows():
    hospital=(row['위도'],row['경도'])
    dist=haversine(patient,hospital,unit='km')
    distance.append(dist)
    
distance_df['거리(km)']=distance

In [225]:
distance_df.head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도,거리(km)
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,2,...,N,N,N,Y,N,N,Y,70.0,원활,11.676284
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9018,35.883009,128.663456,10,4,13,7,...,N,N,N,N,N,N,N,130.0,원활,7.574839
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,0,7,...,Y,Y,Y,Y,Y,Y,Y,0.0,불가,2.110082
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6306,35.853887,128.480130,28,24,0,6,...,Y,Y,Y,Y,N,Y,Y,0.0,불가,9.259961
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,4,...,N,N,N,Y,Y,N,Y,70.0,원활,0.705106


In [229]:
# 거리를 ['2km이내', '5km이내', '10km이내', '10km이상']로 구분
# pd.cut 활용

distance_df['거리구분'] = pd.cut(distance_df['거리(km)'], bins=[0,2,5,10,np.inf], labels=['2km이내', '5km이내', '10km이내', '10km이상'])
distance_df.head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도,거리(km),거리구분
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,2,...,N,N,Y,N,N,Y,70.0,원활,11.676284,10km이상
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9018,35.883009,128.663456,10,4,13,7,...,N,N,N,N,N,N,130.0,원활,7.574839,10km이내
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,0,7,...,Y,Y,Y,Y,Y,Y,0.0,불가,2.110082,5km이내
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6306,35.853887,128.480130,28,24,0,6,...,Y,Y,Y,N,Y,Y,0.0,불가,9.259961,10km이내
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,4,...,N,N,Y,Y,N,Y,70.0,원활,0.705106,2km이내


In [231]:
# 가까운 거리순으로, 응급실 포화도가 원활한 순서로 정렬하기

distance_df.sort_values(['거리(km)','응급실포화도'],inplace=True)

In [232]:
distance_df.head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도,거리(km),거리구분
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,4,...,N,N,Y,Y,N,Y,70.0,원활,0.705106,2km이내
8,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2,7,2,...,Y,N,N,N,N,N,100.0,원활,1.901922,2km이내
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,0,7,...,Y,Y,Y,Y,Y,Y,0.0,불가,2.110082,5km이내
14,A1300001,영남대학교병원,대구광역시 남구 현충로 170 (대명동),053-620-3177,35.847506,128.585131,30,19,0,9,...,Y,Y,N,Y,N,Y,0.0,불가,2.425019,5km이내
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,3,9,...,Y,Y,Y,Y,Y,Y,12.5,혼잡,3.145999,5km이내


### 7) 상황에 따른 병원 데이터 출력하기

 * 상황1) 응급실 수용이 가능한 병원
 * 상황2) 중증질환자 수용이 가능하고 응급실 수용이 가능한 병원
 * 상황3) 중증질환자 수용이 가능하고 응급실 수용이 가능한 병원 중 환자 위치에서 가까운 병원

In [242]:
# 상황1) 응급실 수용이 가능한 병원
# 조건1 : 응급실 포화도가 불가인 병원을 제외하고 출력

condition1 = distance_df[distance_df['응급실포화도']!='불가']
solution_df=condition1
solution_df.head()
#solution_df[condition1].head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도,거리(km),거리구분
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,4,...,N,N,Y,Y,N,Y,70.0,원활,0.705106,2km이내
8,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2,7,2,...,Y,N,N,N,N,N,100.0,원활,1.901922,2km이내
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,3,9,...,Y,Y,Y,Y,Y,Y,12.5,혼잡,3.145999,5km이내
12,A1300034,드림종합병원,대구광역시 남구 대명로 153 (대명동),053-640-8575,35.839635,128.574355,10,3,7,3,...,N,N,Y,Y,Y,Y,70.0,원활,3.330733,5km이내
9,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4,17,4,...,N,N,N,N,N,Y,170.0,원활,3.806643,5km이내


In [243]:
solution_df.columns

Index(['병원코드', '병원명', '주소', '응급연락처', '위도', '경도', '응급실수', '수술실수', '가용응급실수',
       '가용수술실수', '뇌출혈', '뇌경색', '심근경색', '복부손상', '사지접합', '응급투석', '조산산모', '신생아',
       '중증화상', '응급실가용율', '응급실포화도', '거리(km)', '거리구분'],
      dtype='object')

In [246]:
# 상황2) 중증질환자 수용이 가능하고, 응급실 수용이 가능한 병원
# 조건1 : '뇌출혈' 중증질환자 수용 및 수술실 수용 가능한 응급으료기관 조회
# 조건2 : 응급실 포화도가 불가인 병원을 제외하고 출력

mkiosk = '뇌출혈'
condition1 =((distance_df[mkiosk] == 'Y') & (distance_df['가용수술실수']>=1))

condition2 = (distance_df['응급실포화도']!='불가')

distance_df[condition1 & condition2].head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도,거리(km),거리구분
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,3,9,...,Y,Y,Y,Y,Y,Y,12.500000,혼잡,3.145999,5km이내
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5,4,5,...,Y,N,Y,Y,N,Y,36.363636,보통,7.880274,10km이내


In [253]:
# 상황3) 중증질환자 수용이 가능하고, 응급실 수용이 가능한 병원 중 환자 위치에서 가까운 병원
# 조건1 : '중증화상' 중증질환자 수용 및 수술실 수용 가능한 응급으료기관 조회
# 조건2 :응급실 포화도가 불가인 병원을 제외하고
# 정렬 : 환자 위치를 중심으로 가장 가까운 병원부터 출력

# 환자 위치 : 서문시장 내
patient= (35.8690760, 128.5811616)  #Latitude, Longitude

# 중증질환자 수용이 가능하고, 응급실 수용이 가능한 병원
mkiosk = '중증화상'
condition1 = ((distance_df[mkiosk] == 'Y') & (distance_df['가용수술실수']>=1))


condition2 = (distance_df['응급실포화도']!='불가')

distance_df = distance_df[condition1&condition2]

distance_df
# # 환자 위치에서 병원까지의 거리 계산

# distance = []

# for 

# distance_df['거리'] = 

# # 거리를 ['2km이내', '5km이내', '10km이내', '10km이상']로 구분
# # pd.cut 활용

# distance_df['거리구분'] = 

# # 환자 위치에서 가까운 거리순으로, 응급실 포화도가 원활한 순서로 정렬하기

# distance_df


,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도,거리(km),거리구분
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,4,...,N,N,Y,Y,N,Y,70.000000,원활,0.705106,2km이내
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,3,9,...,Y,Y,Y,Y,Y,Y,12.500000,혼잡,3.145999,5km이내
12,A1300034,드림종합병원,대구광역시 남구 대명로 153 (대명동),053-640-8575,35.839635,128.574355,10,3,7,3,...,N,N,Y,Y,Y,Y,70.000000,원활,3.330733,5km이내
9,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4,17,4,...,N,N,N,N,N,Y,170.000000,원활,3.806643,5km이내
13,A1300045,삼일병원,"대구광역시 달서구 월배로 436, 지하1,지상1~11층 (송현동)",053-659-3119,35.832748,128.553855,10,2,7,3,...,Y,N,Y,Y,Y,Y,70.000000,원활,4.730173,5km이내
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5,4,5,...,Y,N,Y,Y,N,Y,36.363636,보통,7.880274,10km이내
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,2,...,N,N,Y,N,N,Y,70.000000,원활,11.676284,10km이상


## 8) 병원 실시간 정보 및 거리를 구하는 파이썬 파일을 작성하기

 * 작성 방법
    * 1) hospital.py로 저장하기
    * 2) 필요한 라이브러리 불러오기
    * 3) 병원 리스트 csv 파일 불러오기
    * 4) 병원 실시간 정보 가져오기
    * 5) 병원 응급실 가용율, 포화도 계산하기
    * 6) 환자 수용 가능한 병원의 거리 구하기 (환자 중증 질환 : 응급투석, 환자 위치 : 대구역 근처 (35.8765167, 128.5972922))
    * 7) 거리 구간 구하기
    * 8) 거리순, 응급실 포화도 순으로 결과 출력하기 (pandas의 to_markdown() 활용)



In [254]:
## tabulate library가 없는 경우 설치 필요
## pandas.to_markdown() 시 tabulate library가 필요함
#!pip install tabulate 

In [266]:
%%writefile hospital.py
# 1) hopistal.py로 저장하기. 해당 명령어는 첫 줄에 써줘야 함

# 2) 필요한 라이브러리 불러오기
import pandas as pd
from urllib.parse import quote
from haversine import haversine
import ssl
from urllib.request import urlopen
import numpy as np

context=ssl.create_default_context()
context.set_ciphers("DEFAULT")


#  [국립중앙의료원 - 전국응급의료기관 조회 서비스] 활용을 위한 개인 일반 인증키(Encoding) 저장

key = "1DQKrkMBrPphK39z1SyNfz5dSesh5VlV%2F6VHMdBgr85Vq%2Fu3YRpKnu9V2phrBdVzYDRdAmZ10rVxDr4kPLbUWw%3D%3D"

# city = 대구광역시, 인코딩 필요
city = quote("대구광역시")

# 3) 병원 리스트 csv 파일 불러오기 (파일명 : daegu_hospital_list.csv)
solution_df = pd.read_csv('daegu_hospital_list.csv')

# 4) 병원 실시간 정보 가져오기

# 응급실 실시간 가용병상 
url_realtime = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire'+'?serviceKey=' + key + '&STAGE1=' + city + '&pageNo=1&numOfRows=100'
result = urlopen(url_realtime, context=context)
emrRealtime = pd.read_xml(result, xpath='.//item')

## 기존 데이터와 병합
tmp=emrRealtime[['hpid', 'hvec', 'hvoc']]
solution_df = pd.merge(solution_df,tmp, on='hpid', how='left')



# 실시간 중증질환자 수용 가능 병원 조회
url_acpt = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire'+'?serviceKey=' + key + '&STAGE1=' + city + '&pageNo=1&numOfRows=100' 
result = urlopen(url_acpt, context=context)
emrAcpt = pd.read_xml(result, xpath='.//item') 
## 갑자기 컬럼명이 바뀜
## hpid가 아닌 dutyName 컬럼명으로 출력됨
## 추후 원복 시 삭제해야할 수 있음

emrAcpt = emrAcpt.rename(columns={"dutyName":"hpid"})

## 기존 데이터오 병합
tmp=emrAcpt[['hpid', 'MKioskTy1', 'MKioskTy2', 'MKioskTy3', 'MKioskTy4', 'MKioskTy5', 'MKioskTy7','MKioskTy8', 'MKioskTy10','MKioskTy11']]
solution_df = pd.merge(solution_df,tmp, on='hpid', how='left')


# 컬럼명 변경하기 

column_change = {  'hpid' : '병원코드',
                   'dutyName' : '병원명',
                   'dutyAddr' : '주소',
                   'dutyTel3' : '응급연락처',
                   'wgs84Lat' : '위도', 
                   'wgs84Lon' : '경도',
                   'hperyn' : '응급실수',
                   'hpopyn' : '수술실수',
                   'hvec' : '가용응급실수',
                   'hvoc' : '가용수술실수',
                   'MKioskTy1' : '뇌출혈', 
                   'MKioskTy2' : '뇌경색',
                   'MKioskTy3' : '심근경색',
                   'MKioskTy4' : '복부손상',
                   'MKioskTy5' : '사지접합',
                   'MKioskTy7' : '응급투석',
                   'MKioskTy8' : '조산산모',
                   'MKioskTy10' : '신생아',
                   'MKioskTy11' : '중증화상',                   
                   }

solution_df = solution_df.rename(columns=column_change)

solution_df = solution_df.replace({"정보미제공": "N"})

# 5) 병원 응급실 가용율, 포화도 계산하기
## 가용 응급실수, 가용 수술실 수가 0보다 작은 경우는 비정상 데이터로 추정
## 0보다 작은 수는 0으로 변경
solution_df.loc[(solution_df['가용응급실수']<0) | (solution_df['가용수술실수']<0), '가용응급실수']=0

# 응급실 가용율 계산 : 가용 응급실수 / 응급실 수
solution_df['응급실가용율'] =  (solution_df['가용응급실수']/solution_df['응급실수'])*100



# 응급실 가용율이 1이 넘는 경우는 1로 대체
solution_df.loc[solution_df['응급실가용율']>=100,'응급실가용율']=100


# 응급실 가용율에 따라 포화도 분류
solution_df['응급실포화도'] = pd.cut(solution_df['응급실가용율'],bins=[-1, 10, 30, 60, np.inf],labels=['불가','혼잡','보통','원활'])



# 6) 환자 수용 가능한 병원의 거리 구하기 (환자 중증 질환 : 응급투석, 환자 위치 : 대구역 근처 (35.8765167, 128.5972922))
# 중증질환자 수용이 가능하고, 응급실 수용이 가능한 병원
mkiosk = '뇌경색'
condition1 =((solution_df[mkiosk] == 'Y') & (solution_df['가용수술실수']>=1))

condition2 = (solution_df['응급실포화도']!='불가')

solution_df=solution_df[condition1 & condition2]


patient= (35.8765167, 128.5972922)  #Latitude, Longitude

# 거리 계산

distance = []
distance_df = solution_df.copy()

for idx, row in distance_df.iterrows():
    hospital=(row['위도'],row['경도'])
    dist=haversine(patient,hospital,unit='km')
    distance.append(dist)
    
distance_df['거리(km)']=distance

# 7) 거리 구간 구하기
distance_df['거리구분'] = pd.cut(distance_df['거리(km)'], bins=[0,2,5,10,np.inf], labels=['2km이내', '5km이내', '10km이내', '10km이상'])

# 응글십가용율, 거리 기준으로 정렬
result=distance_df.sort_values(['거리(km)','응급실가용율'],ascending=[True,False])
print(result.to_markdown())

Overwriting hospital.py


In [267]:
## hostpital.py 실행하여 병원 리스트 출력하기 

!python hospital.py

|    | 병원코드   | 병원명         | 주소                                                    | 응급연락처   |    위도 |    경도 |   응급실수 |   수술실수 |   가용응급실수 |   가용수술실수 | 뇌출혈   | 뇌경색   | 심근경색   | 복부손상   | 사지접합   | 응급투석   | 조산산모   | 신생아   | 중증화상   |   응급실가용율 | 응급실포화도   |   거리(km) | 거리구분   |
|---:|:-----------|:---------------|:--------------------------------------------------------|:-------------|--------:|--------:|-----------:|-----------:|---------------:|---------------:|:---------|:---------|:-----------|:-----------|:-----------|:-----------|:-----------|:---------|:-----------|---------------:|:---------------|-----------:|:-----------|
|  8 | A1300076   | 대구굿모닝병원 | 대구광역시 남구 성당로 224 (대명동)                     | 053-620-9119 | 35.853  | 128.574 |          7 |          2 |              9 |              2 | N        | Y        | Y          | Y          | N          | N          | N          | N        | N          |       100      | 원활           |    3.35222 | 5km이내    |
| 14 | A1300001   | 영남대학교병원 | 